In [1]:
import pandas as pd
import numpy as np
import sys
import re
import requests 
from bs4 import  BeautifulSoup
import json
from tqdm import tnrange, tqdm_notebook
from datetime import datetime
from tqdm import trange,tnrange, tqdm_notebook
import time
import polars as pl

In [2]:
import pyodbc
from requests_html import AsyncHTMLSession
asession = AsyncHTMLSession()
from datetime import date
from dateutil.relativedelta import relativedelta

In [3]:
import plotly as py
import plotly.tools as tls
import plotly.graph_objs as go
from plotly.offline import iplot, plot
py.offline.init_notebook_mode(connected=True)
import ipywidgets as widgets
from ipywidgets import interact, interact_manual

In [4]:
idx = pd.IndexSlice
pd.options.display.float_format = '{:,.3f}'.format

# CRAWL BÁO CÁO MBKE

In [114]:
import os
# dir path for saving file E:/Report/MBKE
dir_path = 'E:/Report/MBKE/'
files = os.listdir(dir_path)
files = [re.findall(r'^\d+',x) for x in files]
files = [item for sublist in files for item in sublist]
files = [int(x) for x in files]

file_list = os.listdir(dir_path)
#create a dataframe with file_list and files
list_file = pd.DataFrame([file_list,files]).T 
list_file.columns = ['filename','id_filename']
# list_file = list(list_file['filename'])
start = max(files)
end = start + 500
min_report = min(files)
#get the number missing in files with range min_report to start
missing = [x for x in range(min_report, start) if x not in files]
quet = [x for x in range(min_report, start)]



In [106]:
def get_report_name(link):
    try:
        r = requests.get(link,timeout=5)
        try:
            with r:
                fname = ''
                if "Content-Disposition" in r.headers.keys():
                    fname = re.findall("filename=(.+)", r.headers["Content-Disposition"])[0]
                else:
                    fname = link.split('/')[-1]
                return fname
        except requests.exceptions.RequestException as e:
            print(e)
    except requests.exceptions.RequestException as e:
        print(e)
        

In [123]:
for i in tnrange(len(quet)):
    url = f'https://www.kelive.com/KimEng/servlet/PDFDownloadViaEmail?source=0&rid={quet[i]}'
    name = get_report_name(url)
    if name != None:
        list_file.loc[i,'id_report'] = str(quet[i])
        list_file.loc[i,'report_name'] = name
    else:
        print(f'{i} - No file')

C:\Users\HP PAVILION\AppData\Local\Temp\ipykernel_32236\1582703808.py:1: TqdmDeprecationWarning:

Please use `tqdm.notebook.trange` instead of `tqdm.tnrange`



  0%|          | 0/4852 [00:00<?, ?it/s]

In [122]:
list_file

,filename,id_filename,id_report,report_name
0,56500_Westports_20220128.pdf,56500,56500,"""56500_Westports_20220128.pdf"""
1,56501_PM-Philippines External Trade Dec 2021 2...,56501,56501,"""56501_PM-Philippines External Trade Dec 2021 ..."
2,56502_India Daily280122.pdf,56502,"56,502.000","""56502_India Daily280122.pdf"""
3,56503_SP-DBS Group 280122.pdf,56503,NaN,NaN
4,56504_RGL-ASEAN X Macro 280122.pdf,56504,NaN,NaN
...,...,...,...,...
4661,61348_MK-Carlsberg Brewery Malaysia 140524.pdf,61348,NaN,NaN
4662,61349_MK-Swift Haulage 140524.pdf,61349,NaN,NaN
4663,61351_KhangDienHouse_20240514.pdf,61351,NaN,NaN
4664,61351_MK-BNM Monetary Policy 090524.pdf,61351,NaN,NaN


In [108]:
link = 'https://www.kelive.com/KimEng/servlet/PDFDownloadViaEmail?source=0&rid='
list_link = pd.DataFrame(columns=['id_file','link'])
#add missing to list_link['id_file']
# list_link['id_file'] = missing
#concat range(start,end) to list_link['id_file']
list_link = pd.concat([list_link,pd.DataFrame(range(start,end),columns=['id_file'])],axis=0,ignore_index=True)
list_link['link'] = link + list_link['id_file'].astype(str)
list_link.reset_index(drop=True)
list_link

,id_file,link
0,61352,https://www.kelive.com/KimEng/servlet/PDFDownl...
1,61353,https://www.kelive.com/KimEng/servlet/PDFDownl...
2,61354,https://www.kelive.com/KimEng/servlet/PDFDownl...
3,61355,https://www.kelive.com/KimEng/servlet/PDFDownl...
4,61356,https://www.kelive.com/KimEng/servlet/PDFDownl...
...,...,...
495,61847,https://www.kelive.com/KimEng/servlet/PDFDownl...
496,61848,https://www.kelive.com/KimEng/servlet/PDFDownl...
497,61849,https://www.kelive.com/KimEng/servlet/PDFDownl...
498,61850,https://www.kelive.com/KimEng/servlet/PDFDownl...


In [107]:
get_report_name(list_link.loc[0,'link'])

'"61115_MK-Cypark Resources 010424.pdf"'

In [100]:
r = requests.get(list_link.loc[0,'link'],timeout=5)
for i in tnrange(len(list_link)):
    try:
        r = requests.get(list_link.loc[i,'link'],timeout=5)
        try:
            with r:
                fname = ''
                if "Content-Disposition" in r.headers.keys():
                    fname = re.findall("filename=(.+)", r.headers["Content-Disposition"])[0]
                else:
                    # print(list_link.loc[i,'id_file'])
                    fname = list_link.loc[i,'link'].split("/")[-1]
            list_link.loc[i,'filename'] = fname[1:-1]
            if list_link.loc[i,'filename'] not in list_file:
                # print('downloading',list_link.loc[i,'filename'])
                with open(dir_path+list_link.loc[i,'filename'], "wb") as f:
                    f.write(r.content)
            else: print('có rồi')

        except requests.exceptions.RequestException as e:
            print(e)
    except requests.exceptions.RequestException as e:
        list_link.loc[i,'status'] = 'ERROR'
        print("Error", list_link.loc[i,'link'])


C:\Users\HP PAVILION\AppData\Local\Temp\ipykernel_32236\4153846540.py:2: TqdmDeprecationWarning:

Please use `tqdm.notebook.trange` instead of `tqdm.tnrange`



  0%|          | 0/500 [00:00<?, ?it/s]

Error https://www.kelive.com/KimEng/servlet/PDFDownloadViaEmail?source=0&rid=61155
Error https://www.kelive.com/KimEng/servlet/PDFDownloadViaEmail?source=0&rid=61178
Error https://www.kelive.com/KimEng/servlet/PDFDownloadViaEmail?source=0&rid=61222
Error https://www.kelive.com/KimEng/servlet/PDFDownloadViaEmail?source=0&rid=61227
Error https://www.kelive.com/KimEng/servlet/PDFDownloadViaEmail?source=0&rid=61239
Error https://www.kelive.com/KimEng/servlet/PDFDownloadViaEmail?source=0&rid=61258
Error https://www.kelive.com/KimEng/servlet/PDFDownloadViaEmail?source=0&rid=61289
Error https://www.kelive.com/KimEng/servlet/PDFDownloadViaEmail?source=0&rid=61296
Error https://www.kelive.com/KimEng/servlet/PDFDownloadViaEmail?source=0&rid=61306
Error https://www.kelive.com/KimEng/servlet/PDFDownloadViaEmail?source=0&rid=61320
Error https://www.kelive.com/KimEng/servlet/PDFDownloadViaEmail?source=0&rid=61335
Error https://www.kelive.com/KimEng/servlet/PDFDownloadViaEmail?source=0&rid=61350
6135

OSError: [Errno 22] Invalid argument: 'E:/Report/MBKE/DFDownloadViaEmail?source=0&rid=6135'

In [69]:
list_link.loc[3,'link']

'https://www.kelive.com/KimEng/servlet/PDFDownloadViaEmail?source=0&rid=56594'

# HOWARD MARKS - OAKTREE CAPITAL

In [159]:
pdf_folder = os.path.expanduser("~/Desktop/SuperInvestor/Howard Marks/")
output_pdf = 'HowardMarks-Memo-1990_2023.pdf'
pdf_files = []
for filename in os.listdir(pdf_folder):
    if filename.endswith('.pdf'):
        pdf_files.append(os.path.join(pdf_folder, filename))
if not pdf_files:
    print("No PDF files found in the directory.")
else:
    pdf_merger = PyPDF2.PdfMerger()
    for pdf_file in pdf_files:
        pdf_merger.append(pdf_file)
    pdf_merger.write(output_pdf)
    pdf_merger.close()

    print(f"Merged PDF saved as '{output_pdf}'")

Merged PDF saved as 'HowardMarks-Memo-1990_2023.pdf'


# Emeth Value Capital

In [160]:
pdf_folder = os.path.expanduser("~/Desktop/SuperInvestor/Emeth Value Capital/")
output_pdf = 'Emeth Value Capital-2018_2023.pdf'
pdf_files = []
for filename in os.listdir(pdf_folder):
    if filename.endswith('.pdf'):
        pdf_files.append(os.path.join(pdf_folder, filename))
if not pdf_files:
    print("No PDF files found in the directory.")
else:
    pdf_merger = PyPDF2.PdfMerger()
    for pdf_file in pdf_files:
        pdf_merger.append(pdf_file)
    pdf_merger.write(output_pdf)
    pdf_merger.close()

    print(f"Merged PDF saved as '{output_pdf}'")

Merged PDF saved as 'Emeth Value Capital-2018_2023.pdf'


# St James Investment Company

In [33]:
pdf_folder = os.path.expanduser("~/Desktop/SuperInvestor/St James Investment Company/")
output_pdf = 'St James Investment Company_2014-2023.pdf'
pdf_files = []
for filename in os.listdir(pdf_folder):
    if filename.endswith('.pdf'):
        pdf_files.append(os.path.join(pdf_folder, filename))
if not pdf_files:
    print("No PDF files found in the directory.")
else:
    pdf_merger = PyPDF2.PdfMerger()
    for pdf_file in pdf_files:
        pdf_merger.append(pdf_file)
    pdf_merger.write(output_pdf)
    pdf_merger.close()

    print(f"Merged PDF saved as '{output_pdf}'")

Merged PDF saved as 'St James Investment Company_2014-2023.pdf'


In [34]:
pdf_folder = os.path.expanduser("~/Desktop/SuperInvestor/TaoValue/")
output_pdf = 'TaoValue_2017-2021.pdf'
pdf_files = []
for filename in os.listdir(pdf_folder):
    if filename.endswith('.pdf'):
        pdf_files.append(os.path.join(pdf_folder, filename))
if not pdf_files:
    print("No PDF files found in the directory.")
else:
    pdf_merger = PyPDF2.PdfMerger()
    for pdf_file in pdf_files:
        pdf_merger.append(pdf_file)
    pdf_merger.write(output_pdf)
    pdf_merger.close()

    print(f"Merged PDF saved as '{output_pdf}'")

Merged PDF saved as 'TaoValue_2017-2021.pdf'


In [35]:
pdf_folder = os.path.expanduser("~/Desktop/SuperInvestor/Righ Tail Capital/")
output_pdf = 'Right Tail Capital_2022-2023.pdf'
pdf_files = []
for filename in os.listdir(pdf_folder):
    if filename.endswith('.pdf'):
        pdf_files.append(os.path.join(pdf_folder, filename))
if not pdf_files:
    print("No PDF files found in the directory.")
else:
    pdf_merger = PyPDF2.PdfMerger()
    for pdf_file in pdf_files:
        pdf_merger.append(pdf_file)
    pdf_merger.write(output_pdf)
    pdf_merger.close()

    print(f"Merged PDF saved as '{output_pdf}'")

Merged PDF saved as 'Righ Tail Capital_2022-2023.pdf'


In [55]:
pdf_folder = os.path.expanduser("~/Desktop/SuperInvestor/Lindsell Train/")
output_pdf = 'Righ Tail Capital_2022-2023.pdf'
pdf_files = []
# for filename in os.listdir(pdf_folder):
#     if filename.endswith('.pdf'):
#         pdf_files.append(os.path.join(pdf_folder, filename))
# if not pdf_files:
#     print("No PDF files found in the directory.")
# else:
#     pdf_merger = PyPDF2.PdfMerger()
#     for pdf_file in pdf_files:
#         pdf_merger.append(pdf_file)
#     pdf_merger.write(output_pdf)
#     pdf_merger.close()

#     print(f"Merged PDF saved as '{output_pdf}'")

# HaydenCapital

In [144]:
import os
dir_path = os.path.expanduser("~/Desktop/SuperInvestor/Hayden Capital/")
file_list = os.listdir(dir_path)
list_file = pd.DataFrame(file_list, columns=['filename'])
list_file = list(list_file['filename'])
x['check'] = x['name'].isin(list_file)

In [145]:
x = x[x['check']==False].reset_index()

In [146]:
def download_pdf(url, save_path):
    headers = {"Accept": "application/pdf",
              "User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"}  # Specify that you accept PDF format

    response = requests.get(url, headers=headers)
    
    if response.status_code == 200:
        with open(save_path, 'wb') as pdf_file:
            pdf_file.write(response.content)
        print(f"PDF downloaded successfully to {save_path}")
    elif response.status_code == 406:
        print("The server cannot provide a response in the format you accept (PDF).")
    else:
        print(f"Failed to download PDF. Status code: {response.status_code}")

In [151]:
desktop_path = os.path.expanduser("~/Desktop/Superinvestor/Hayden Capital/")
for i in tnrange(len(x)):
    save_path = os.path.join(desktop_path, x.loc[i,'name'])
    download_pdf(x.loc[i,'link'], save_path)

C:\Users\HP PAVILION\AppData\Local\Temp\ipykernel_4824\277642095.py:2: TqdmDeprecationWarning:

Please use `tqdm.notebook.trange` instead of `tqdm.tnrange`



  0%|          | 0/22 [00:00<?, ?it/s]

Failed to download PDF. Status code: 404
Failed to download PDF. Status code: 404
Failed to download PDF. Status code: 404
Failed to download PDF. Status code: 404
Failed to download PDF. Status code: 404
Failed to download PDF. Status code: 404
Failed to download PDF. Status code: 404
Failed to download PDF. Status code: 404
Failed to download PDF. Status code: 404
Failed to download PDF. Status code: 404
Failed to download PDF. Status code: 404
Failed to download PDF. Status code: 404
Failed to download PDF. Status code: 404
PDF downloaded successfully to C:\Users\HP PAVILION/Desktop/Superinvestor/Hayden Capital/Hayden-Capital-Quarterly-Letter-2017-Q2.pdf
PDF downloaded successfully to C:\Users\HP PAVILION/Desktop/Superinvestor/Hayden Capital/Hayden-Capital-Quarterly-Letter-2017-Q3.pdf
PDF downloaded successfully to C:\Users\HP PAVILION/Desktop/Superinvestor/Hayden Capital/Hayden-Capital-Quarterly-Letter-2017-Q4.pdf
PDF downloaded successfully to C:\Users\HP PAVILION/Desktop/Superinv

In [152]:
pdf_folder = os.path.expanduser("~/Desktop/SuperInvestor/Hayden Capital/")
output_pdf = 'Hayden_2017-2023.pdf'
pdf_files = []
for filename in os.listdir(pdf_folder):
    if filename.endswith('.pdf'):
        pdf_files.append(os.path.join(pdf_folder, filename))
if not pdf_files:
    print("No PDF files found in the directory.")
else:
    pdf_merger = PyPDF2.PdfMerger()
    for pdf_file in pdf_files:
        pdf_merger.append(pdf_file)
    pdf_merger.write(output_pdf)
    pdf_merger.close()

    print(f"Merged PDF saved as '{output_pdf}'")

Merged PDF saved as 'Hayden_2017-2023.pdf'
